In [1]:
import torch, torchvision
import torchvision.transforms.functional as fn
torch.manual_seed(0)

import os
import numpy as np
import matplotlib.pyplot as plt
import sklearn

In [2]:
#Update os
path = "/home/kishen/documents/python_projects/dogs_v_cats/"
os.chdir(path)
from src.dataset import ImageDataset

Preprocessing

In [3]:
#Create labels: cats=0, dogs=1
labels = np.zeros(25_000)
labels[12_500:] =  1

In [4]:
#create augmentations for ImageDataset
path = "/home/kishen/documents/python_projects/dogs_v_cats/"
os.chdir(path)
from src.dataset import ImageDataset

transformations = torch.nn.Sequential(
        torchvision.transforms.Resize((224,224)),
        torchvision.transforms.RandomPerspective(p=.25),
        torchvision.transforms.RandomHorizontalFlip(p=.25)

)

#Build training dataset
training_data = ImageDataset(image_paths= "/home/kishen/documents/python_projects/dogs_v_cats/data/train/",
                            targets=labels,
                            augmentations= transformations)



In [5]:
#create dataloader for training data
from torch.utils.data import DataLoader

loader_params = {"batch_size": 16,
                 "shuffle": True,
                 "num_workers": 16}
                 
train_loader = DataLoader(training_data, **loader_params)

**Training - BaseConvNet**

In [6]:
#init model and optimizer
from src.models import BaseConvNet, train, accuracy

model = BaseConvNet()

optimizer_params={'lr':.0001,
             'momentum':.9}

optimizer = torch.optim.SGD(model.parameters(), **optimizer_params)

#move model to gpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
print(device)

#set train params
train_params = {
'model': model,
'loader' : train_loader, 
'epochs': 20, 
'criterion':torch.nn.BCEWithLogitsLoss(), 
'optimizer': optimizer
}



cuda


In [7]:
# train(**train_params)

In [8]:
# torch.save(model.state_dict(), "/home/kishen/documents/python_projects/dogs_v_cats/models/base_model")

Revisit base model and update padding, kernel, stride, etc. to better maintain image volume

**Training - ConvNet2_0**

In [9]:
#init ConvNet1_0
from src.models import ConvNet1_0

model2 = ConvNet1_0()
model2.to(device)
print(device)

#set train params
train_params2 = {
'model': model2,
'loader' : train_loader, 
'epochs': 20, 
'criterion':torch.nn.BCEWithLogitsLoss(), 
'optimizer': optimizer
}

In [ ]:
train(**train_params2)